# Run model

In [1]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('./trained_model')
model.eval()

device = torch.device('cpu')
model = model.to(device)

category_map = {
    0: 'Normal',
    1: 'Depression',
    2: 'Suicidal',
    3: 'Anxiety',
    4: 'Stress',
    5: 'Bi-Polar',
    6: 'Personality Disorder'
}

def predict_text(text):
    inputs = tokenizer(
        text,
        truncation=True,
        padding='max_length',
        max_length=128,
        return_tensors='pt'
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
        predicted_class = torch.argmax(predictions, dim=-1).item()
        confidence = predictions[0][predicted_class].item()

    return category_map[predicted_class], confidence

In [2]:
text = "I feel overwhelmed and can't cope with daily tasks"
category, confidence = predict_text(text)
print(f"Prediction: {category} (confidence: {confidence:.2%})")

Prediction: Depression (confidence: 96.78%)


In [4]:
text = "I am happy and stressed"
category, confidence = predict_text(text)
print(f"Prediction: {category} (confidence: {confidence:.2%})")

Prediction: Normal (confidence: 92.57%)


In [5]:
text = "I am suicidal and need help"
category, confidence = predict_text(text)
print(f"Prediction: {category} (confidence: {confidence:.2%})")

Prediction: Suicidal (confidence: 90.31%)
